In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('../../input/melbourne_housing_snapshot/melb_data.csv')

y = data.Price

X = data.drop(['Price'], axis=1)

X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y,
                                                          train_size=0.8, test_size=0.2)

categorical_cols = [col for col in X_train_full.columns if X_train_full[col].nunique() < 10 
                       and X_train_full[col].dtype == 'object']
numerical_cols = [col for col in X_train_full.columns if X_train_full[col].dtype in ['int64', 'float64']]

my_cols = categorical_cols + numerical_cols

X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [5]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
2482,h,S,Western Metropolitan,5,9.4,3041.0,5.0,3.0,2.0,463.0,196.0,1930.0,-37.73790,144.90220,1308.0
5742,u,PI,Southern Metropolitan,2,1.2,3006.0,2.0,2.0,2.0,0.0,73.0,2014.0,-37.82290,144.96120,8400.0
3184,u,VB,Southern Metropolitan,3,4.6,3122.0,3.0,2.0,1.0,0.0,196.0,2010.0,-37.82350,145.02900,11308.0
8853,h,S,Southern Metropolitan,2,5.4,3101.0,2.0,1.0,2.0,901.0,NaN,NaN,-37.79719,145.03177,10331.0
9271,u,SP,Southern Metropolitan,2,11.4,3204.0,2.0,1.0,1.0,0.0,72.0,2012.0,-37.90335,145.03718,3578.0


In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [9]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

In [10]:
from sklearn.metrics import mean_absolute_error

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 164888.0380305421
